In [2]:
import pandas as pd
source = pd.read_excel("products.xlsx")
target = pd.read_excel("products_FR_V2.xlsx")

In [3]:
source.columns == target.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True])

In [4]:
import pandas as pd

# Load the uploaded files
products_df = pd.read_excel('products.xlsx')
products_fr_v2_df = pd.read_excel('products_FR_V2.xlsx')

# Rename columns in products_df by adding 'nl_' prefix
products_df.columns = ['nl_' + col for col in products_df.columns]

# Merge the two DataFrames
merged_df = pd.concat([products_df, products_fr_v2_df], axis=1)


In [5]:
merged_df.head()

,nl_id,nl_Pack unit plural,nl_Pack unit,nl_Meta title,nl_Keywords,nl_Description,nl_Meta description,nl_Custom search keywords,nl_name,id,Pack unit plural,Pack unit,Meta title,Keywords,Description,Meta description,Custom search keywords,name
0,51bd0d999c7a4fd28e1eee231f2fee50,NaN,NaN,NaN,NaN,<h2>Ade Keukenweegschaal Ramona - Elektronisch...,NaN,NaN,Ade Keukenweegschaal Ramona - Elektronisch - B...,51bd0d999c7a4fd28e1eee231f2fee50,NaN,NaN,NaN,NaN,<h2>Balance de cuisine Ade Ramona - Électroniq...,NaN,NaN,Balance de cuisine Ade Ramona - Électronique -...
1,2827dc8808df485d9f676f98325daadc,NaN,NaN,Ade Digitale Keukenweegschaal Cleo - D&L,NaN,<h2>Ade Keukenweegschaal Cleo - Elektronisch -...,Ade Digitale Keukenweegschaal Cleo,"[""weegschaal""]",Ade Digitale Keukenweegschaal Cleo,2827dc8808df485d9f676f98325daadc,NaN,NaN,Balance de cuisine numérique Ade Cleo - D&L,NaN,<h2>Balance de cuisine Ade Cleo - Électronique...,Balance de cuisine numérique Ade Cleo,"[""balance""]",Balance de cuisine numérique Ade Cleo
2,d657a7202ef54ff9a79b7ab44ef6780e,NaN,NaN,Ade Keukenweegschaal Julia - Elektronisch - Wi...,NaN,<h2>Ade Keukenweegschaal Julia - Elektronisch ...,Ade Keukenweegschaal Julia - Elektronisch - Wit,"[""keuken weegschaal""]",Ade Keukenweegschaal Julia - Elektronisch - Wit,d657a7202ef54ff9a79b7ab44ef6780e,NaN,NaN,Balance de cuisine Ade Julia - Électronique - ...,NaN,<h2>Balance de cuisine Ade Julia - Électroniqu...,Balance de cuisine Ade Julia - Électronique - ...,"[""balance de cuisine""]",Balance de cuisine Ade Julia - Électronique - ...
3,2bb95019f5c240a0b9e75fe37eb2fb57,NaN,NaN,NaN,NaN,<h2>Ade Keukenweegschaal Angelina - Elektronis...,NaN,NaN,Ade Keukenweegschaal Angelina - Elektronisch -...,2bb95019f5c240a0b9e75fe37eb2fb57,NaN,NaN,NaN,NaN,<h2>Balance de cuisine Ade Angelina - Électron...,NaN,NaN,Balance de cuisine Ade Angelina - Électronique...
4,f2d985cfd7134873842d1536adfd4142,NaN,NaN,NaN,NaN,<h2>Ade Keukenweegschaal Denise - Elektronisch...,NaN,NaN,Ade Keukenweegschaal Denise - Elektronisch - Z...,f2d985cfd7134873842d1536adfd4142,NaN,NaN,NaN,NaN,<h2>Balance de cuisine Ade Denise - Électroniq...,NaN,NaN,Balance de cuisine Ade Denise - Électronique -...


In [3]:
merged_df.shape

(4207, 18)

In [21]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o",temperature=0)

 ········


In [7]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='ciao!', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e3da7188-815c-4242-8173-47a495fd4844-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23})

In [8]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [9]:
chain = model | parser
chain.invoke(messages)

'ciao!'

In [11]:
from langchain_core.prompts import ChatPromptTemplate
system_template = "Translate the following into {language}:"

In [12]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)
result = prompt_template.invoke({"language": "italian", "text": "hi"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:'), HumanMessage(content='hi')])

In [23]:
system_template = """You are getting a translation from dutch to french, 
however the translation software has made mistakes in putting html back into the target language. 
You need to keep the translation text itself intact, 
but need to to fix the html so that the html in the target matches the html in the source. 
the information will be: ID: id, SOURCE_NAME: source text, 
                         TARGET_NAME: target name with possible errors in name, 
                         SOURCE_DESCRIPTION: source description,
                         TARGET_DESCRIPTION: target description with possible errors in html. 
you give back the corrected target text. if it is correct, just give back correct, else the corrected output 
The source html is the truth.
after you have fixed the html, you need to check that the translated text is exactly the same.
output format in json with fields: id,
                                  skipped_because_correct,
                                  source_name, 
                                  target_name, 
                                  corrected_target_name,  
                                  verified_translation_is_equal_to_target_name, 
                                  text_without_html_is_equal_to_target_name,
                                  source_description,
                                  target_description,
                                  corrected_target_description,
                                  verified_translation_is_equal_to_target_description,
                                  text_without_html_is_equal_to_target_description
if the html is the same in name and description, you can just set the skipped_because_correst to True and set other fields to None but include them in the other fields
                                  
"""
from pydantic import BaseModel
from typing import Optional

class ProductTranslation(BaseModel):
    id: str
    skipped_because_correct: Optional[bool]
    source_name: Optional[str]
    target_name: Optional[str]
    corrected_target_name: Optional[str]
    verified_translation_is_equal_to_target_name: Optional[bool]
    text_without_html_is_equal_to_target_name: Optional[bool]
    source_description: Optional[str]
    target_description: Optional[str]
    corrected_target_description: Optional[str]
    verified_translation_is_equal_to_target_description: Optional[bool]
    text_without_html_is_equal_to_target_description: Optional[bool]
    
from langchain_core.output_parsers import JsonOutputParser
parser = JsonOutputParser(pydantic_object=ProductTranslation)
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "ID:{id}, SOURCE_NAME:{source_name}, TARGET_NAME:{target_name}, SOURCE_DESCRIPTION:{source_description}, TARGET_DESCRIPTION:{target_description}")]
)

output = []
chain = prompt_template | model | parser
for i, row in merged_df[:10].iterrows():
    result = chain.invoke({"id":row['id'],
                           "source_name": row['nl_name'], 
                           "target_name": row['name'],
                           "source_description": row['nl_Description'], 
                           "target_description": row['Description'],
                          })
    output.append(result)
    print(row['id'])

51bd0d999c7a4fd28e1eee231f2fee50
2827dc8808df485d9f676f98325daadc
d657a7202ef54ff9a79b7ab44ef6780e
2bb95019f5c240a0b9e75fe37eb2fb57
f2d985cfd7134873842d1536adfd4142
864d02fe45f6450dba7807ef133f916d
16f8aacd5a2b4ab68997fe59a83ca5b4
efdfb41415ae48da8acd86e02e93723f
40c5262cafdf470d9169cb27851ec851
52190055307741a8a571b9cd23983953


In [27]:
output_df = pd.DataFrame(output)

merged_df_with_output = pd.merge(merged_df, output_df, on='id', how='left')

# Save the merged DataFrame to a file if needed
merged_df_with_output.to_excel('merged_products_with_output.xlsx', index=False)